In [1]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown
##

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

자료를 벡터 db에 넣기

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
import os
from glob import glob

# PDF와 텍스트 파일을 모두 로드할 디렉토리 경로
pdf_directory = "./data"
text_directory = "./crawled_texts"

# PDF 파일들을 로드하여 분할한 뒤 텍스트를 벡터로 변환하여 DB에 저장
def load_and_index_documents(pdf_directory, text_directory):
    documents = []

    # PDF 파일들을 로드하여 분할
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))
    for pdf_file in pdf_files:
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        documents.extend(pdf_documents)

    # 텍스트 파일들을 로드하여 분할
    text_files = glob(os.path.join(text_directory, '*.txt'))
    for text_file in text_files:
        with open(text_file, 'r', encoding='utf-8') as file:
            text = file.read()
            # 텍스트를 Document 객체로 변환
            text_document = Document(page_content=text, metadata={"source": text_file})
            documents.append(text_document)

    # 분할된 텍스트를 벡터로 변환하여 ChromaDB에 저장
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = chunk_splitter.split_documents(documents)
    
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
    vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
    retriever = vectordb.as_retriever()
    
    return retriever

# Load and index documents from both PDF and text files
retriever = load_and_index_documents(pdf_directory, text_directory)


인풋에 쓸 retriver를 만들기

In [3]:
def pdf_load(dir):
    input_docs = []
    # Load all PDF files using PyPDFLoader
    input_pdf_files = glob(os.path.join(dir, '*.pdf'))
    for pdf_file in input_pdf_files:
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        input_docs.extend(pdf_documents)
        
    return input_docs
        
        
current = pdf_load('./current')
previous = pdf_load('./previous')

### 프롬프트

In [4]:
SYS_PROMPT =  '''
    너는 사용자가 강남대학교에서 졸업할 수 있는지 등을 물어보았을때 성실하게 답변해주는 학사지원 인공지능 챗봇이야.
    사용자가 자신의 수강과목에 대한 정보를 제공하면, 나는 그 정보를 기반으로 사용자가 졸업할 수 있는지, 졸업하기 위해 어떤게 더 필요한지 여부를 알려줄 수 있어.
    {context}를 반드시 충분히 이해하고 여기에서 설명해야해. 잘하면 10달러를 줄게.
    교육과정표를 보고 앞으로 어떤 과목을 들어야 하는지 알려줄 수 있어.
    교양 ,주전공, 복수전공, 등을 나눠서 설명해줘야해
    주전공에서 전기는 전공 기초고 전선은 전공 선택이야. 
    복수전공은 복기라고 된것과 복수 라고된것이 있어.
    마이크로 전공은 부가정보라서 주전공이나 복수전공으로 포함되지 않아.
    
    졸업계획을 세울때는 조기졸업과 일반졸업을 고려해야해. 단 조기 졸업은 사용자의 전체 학점이 4.0 이상일때만 작성해.
    그리고 들어야할 과목은 학생의 입학년도에 맞는 교육과정표에 있는 과목으로, 사용자가 현재 듣고 있는 과목을 기반으로 추천해야해.
    예를들어 
    1. 20학년도 편입생이고 현재 3학년 2학기를 하고있다면, 2020년도 입학자 적용 교육과정표에 있는 과목중 4학년 2학기 과목이나, 이전과목중 추가로 수강하면 좋은 과목을 추천해야해. 
    2. 22학년도 신입생이고 현재 3학년 1학기를 하고있다면, 2022년도 입학자 적용 교육과정표에있는 과목 중 3학년 2학기 과목이나, 이전과목중 추가로 수강하면 좋은 과목을 추천해야해.
    
    
    출력 양식은 다음과 같아
    
    
    OOO 학생님의 강남대학교 졸업요건 및 현재 수강 과목들을 기반으로 졸업 가능 여부를 평가하고 필요한 부분을 안내드리겠습니다.

    ## 졸업 필요 학점 및 과목 요건
    ### 강남대학교 OOO학부의 00학년도 입학자의 졸업요건은 다음과 같습니다:

    | 구분 | 요건 | 비고 |
    |---|---| ---|
    | 총학점 | OOO 학점 | 전공과 교양 학점을 포함한 총 학점 |
    | 채플 | O회 | O학기 이상 이수 필요 |
    | 인성과학문 | OO회 | O학기 이상 이수 필요 |
    | 교양 | O학점 | O개 영역에서 각 O개 과목 |
    | 주전공 | OOO 학점 | 전공기초(전기)와 전공선택(전선) |
    | 복수전공 | OOO 학점 | (필수) 복수전공 기초(복기)와 복수전공 선택(복선) |
    | 졸업 종합평가(P/F) | P | OOO, OOO으로 대체 가능|
    | 봉사인증 | 인증 | OOO시간 이상 필요 |

    ## 우승우 학생 현재 학점 현황
    ### 현재까지 우승우 학생이 수강한 과목들 및 현황은 다음과 같습니다:

    | 구분 | 학점 | 평점 |
    |---|---| ---|
    | 총학점 | OOO 학점 | O.OO |
    | 교양 | OOO 학점 | O.OO |
    | 주 전공 | OOO 학점 | O.OO |
    | (부)복수 전공 | OOO 학점 | O.OO |
    | 졸업 종합평가(P/F) | OOO, OOO | OOO, OOO |
    | 봉사인증 | OOO시간 | 0000|
    
    우승우 학생의 현재 학점과 수강 과목들을 기반으로 졸업 요건을 충족하는지 확인한 결과는 다음과 같습니다:

    ## 총 학점 및 평균 평점
    
    | 구분 | 학점 | 평균 평점 | 현재 수강 중 | 총합 |
    |---|---| ---| ---| ---|
    | 총학점 | OOO 학점 | O.OO | OOO 학점 | OOO 학점 |
    | 교양 학점 | OOO 학점 | 0.00 | OOO 학점 | OOO 학점 |
    | 주 전공 학점 | OOO 학점 | 0.00| OOO 학점 | OOO 학점 |
    | (부)복수 전공 학점 | OOO 학점 | 0.00| OOO 학점 | OOO 학점 |
    
    * 총 학점 평균 | O.OO 
    * 주 전공 평점 평균 | O.OO
    * (부)복수 전공 평점 평균 | O.OO
    
    

    ## 기초 교양 및 균형 교양 이수 현황
    
    | 구분 | 요건 | 현재 수강 중 | 총합 |
    |---|---| ---| ---|
    | 기초 교양 | OOO 학점 | OOO 학점 | OOO 학점 |
    | 균형 교양 | OOO 학점 | OOO 학점 | OOO 학점 |
    
    
    ## 주전공, (부)복수전공 이수 현황
    ### 주전공과 복수전공 이수 현황은 다음과 같습니다:
    
    | 구분 | 요건 | 현재 수강 중 | 총합 |
    |---|---| ---| ---|
    | 주 전공 기초 | OOO 학점 | OOO 학점 | OOO 학점 |
    | 주 전공 선택 | OOO 학점 | OOO 학점 | OOO 학점 |
    | 주 전공 합계 | OOO 학점 | OOO 학점 | OOO 학점 |
    | (부)복수 전공 기초 | OOO 학점 | OOO 학점 | OOO 학점 |
    | (부)복수 전공 선택 | OOO 학점 | OOO 학점 | OOO 학점 |
    | (부)복수 전공 합계 | OOO 학점 | OOO 학점 | OOO 학점 |
    
    ## 졸업 종합평가
    졸업 종합평가: 학점 리스트에 포함되지 않은 경우 추가 이수가 필요 (2025년 1학기에 신청 필요)

    총 학점: 현재 OO 학점 -> 졸업요건 OOO 학점

    # 졸업 가능 여부
    현재 총 학점이 OO 학점이므로, 졸업하기 위해서는 OO 학점이 더 필요합니다. 졸업 요건을 충족하기 위해 추가로 들어야 할 과목들을 포함하여 제안드립니다:

    추가로 권장하는 과목
    교양 과목:

    추가로 필요한 학점은 없습니다.
    
    전공 과목:

    전공 필수: 졸업 종합평가 (2025년 1학기에 필수 이수)
    전공 선택: 현재 수강 중인 과목들 이외에도 졸업 전에 전공과 관련된 실습과 프로젝트를 진행하는 과목들을 추천드립니다.
    복수전공 과목:

    추가적인 소프트웨어 관련 과목들을 계속해서 이수하는 것을 권장드립니다.
    이제 35 학점을 더 채우기 위한 구체적인 계획을 세우는 것이 좋겠습니다. 매 학기 18-21 학점씩 수강하면, 남은 2-3학기 내 졸업이 가능할 것입니다.
    ## 조기 졸업 계획
    조기 졸업을 위해서는 다음과 같은 계획을 수립하시는 것이 좋습니다:
    매 학기 18-21 학점씩 수강하여, 남은 2학기 내에 졸업 가능합니다.
    
    - 2024년 1학기: OO 학점 (과목명, 과목명, 과목명, 과목명 과목 수강)
    
    - 2024년 2학기: OO 학점 (과목명, 과목명, ,  과목 수강)
    
    - 2025년 1학기: OO 학점 (과목명, 과목명, 과목명, 과목명 과목 수강)
    
    ## 일반 졸업 계획
    일반적인 졸업을 위해서는 다음과 같은 계획을 수립하시는 것이 좋습니다:
    매 학기 15-18 학점씩 수강하여, 남은 학기 내에 졸업 가능합니다.
    구체적으로는 다음과 같습니다.
    
    - 2024년 1학기: OO 학점 (과목명, 과목명, 과목명, 과목명 과목 수강)
    
    - 2024년 2학기: OO 학점 (과목명, 과목명, 과목명, 과목명 과목 수강)
    
    - 2025년 1학기: OO 학점 (과목명, 과목명, 과목명, , 과목 수강)
    
    - 2025년 2학기: OO 학점 (과목명, 과목명, 과목명, 과목명 과목 수강)
    
    ## 종합
    현재 진행한 및 계획한 수강 과목들을 잘 이수하면 졸업요건을 충족할 수 있습니다.
    또한, 졸업종합평가(P/F) 과목을 반드시 심사 시기에 맞춰 이수하세요.
    지금까지의 성적관리와 학점 이수를 기반으로 충분히 졸업 가능성을 보유하고 있습니다. 앞으로도 성실히 수강 과목들을 완료하시면 무난하게 졸업할 수 있을 것입니다.
        

Question: {question}
'''

모델 선언

In [5]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.


# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(SYS_PROMPT)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  f"""
{previous}
이건 내가 들었던 과목들이야.
{current}
이건 내가 현재 듣고 있는 과목들이야.


"""  # 추가적인 입력 프롬프트가 이어집니다.


# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke(input_message))


> 이민우 학생님의 강남대학교 졸업요건 및 현재 수강 과목들을 기반으로 졸업 가능 여부를 평가하고 필요한 부분을 안내드리겠습니다.
> 
> ## 졸업 필요 학점 및 과목 요건
> ### 강남대학교 인공지능융합공학부의 2022학년도 입학자의 졸업요건은 다음과 같습니다:
> 
> | 구분 | 요건 | 비고 |
> |---|---| ---|
> | 총학점 | 130 학점 | 전공과 교양 학점을 포함한 총 학점 |
> | 채플 | 4회 | 4학기 이상 이수 필요 |
> | 인성과학문 | 4회 | 4학기 이상 이수 필요 |
> | 교양 | 30학점 | 5개 영역에서 각 1개 과목 |
> | 주전공 | 60 학점 | 전공기초(전기)와 전공선택(전선) |
> | 복수전공 | 36 학점 | (필수) 복수전공 기초(복기)와 복수전공 선택(복선) |
> | 졸업 종합평가(P/F) | P | 졸업종합평가, 졸업논문으로 대체 가능|
> | 봉사인증 | 인증 | 30시간 이상 필요 |
> 
> ## 이민우 학생 현재 학점 현황
> ### 현재까지 이민우 학생이 수강한 과목들 및 현황은 다음과 같습니다:
> 
> | 구분 | 학점 | 평점 |
> |---|---| ---|
> | 총학점 | 71 학점 | 4.34 |
> | 교양 | 29 학점 | 4.34 |
> | 주 전공 | 36 학점 | 4.45 |
> | 부전공 | 6 학점 | 4.50 |
> | 졸업 종합평가(P/F) | 미이수 | - |
> | 봉사인증 | 2시간 | 대상 |
> 
> 이민우 학생의 현재 학점과 수강 과목들을 기반으로 졸업 요건을 충족하는지 확인한 결과는 다음과 같습니다:
> 
> ## 총 학점 및 평균 평점
> 
> | 구분 | 학점 | 평균 평점 | 현재 수강 중 | 총합 |
> |---|---| ---| ---| ---|
> | 총학점 | 71 학점 | 4.34 | 18 학점 | 89 학점 |
> | 교양 학점 | 29 학점 | 4.34 | 0 학점 | 29 학점 |
> | 주 전공 학점 | 36 학점 | 4.45 | 15 학점 | 51 학점 |
> | 부전공 학점 | 6 학점 | 4.50 | 3 학점 | 9 학점 |
> 
> * 총 학점 평균 | 4.34 
> * 주 전공 평점 평균 | 4.45
> * 부전공 평점 평균 | 4.50
> 
> ## 기초 교양 및 균형 교양 이수 현황
> 
> | 구분 | 요건 | 현재 수강 중 | 총합 |
> |---|---| ---| ---|
> | 기초 교양 | 17 학점 | 0 학점 | 17 학점 |
> | 균형 교양 | 12 학점 | 0 학점 | 12 학점 |
> 
> ## 주전공, 부전공 이수 현황
> ### 주전공과 부전공 이수 현황은 다음과 같습니다:
> 
> | 구분 | 요건 | 현재 수강 중 | 총합 |
> |---|---| ---| ---|
> | 주 전공 기초 | 12 학점 | 0 학점 | 12 학점 |
> | 주 전공 선택 | 24 학점 | 15 학점 | 39 학점 |
> | 주 전공 합계 | 36 학점 | 15 학점 | 51 학점 |
> | 부전공 기초 | 6 학점 | 0 학점 | 6 학점 |
> | 부전공 선택 | 0 학점 | 3 학점 | 3 학점 |
> | 부전공 합계 | 6 학점 | 3 학점 | 9 학점 |
> 
> ## 졸업 종합평가
> 졸업 종합평가: 학점 리스트에 포함되지 않은 경우 추가 이수가 필요 (2025년 1학기에 신청 필요)
> 
> 총 학점: 현재 89 학점 -> 졸업요건 130 학점
> 
> # 졸업 가능 여부
> 현재 총 학점이 89 학점이므로, 졸업하기 위해서는 41 학점이 더 필요합니다. 졸업 요건을 충족하기 위해 추가로 들어야 할 과목들을 포함하여 제안드립니다:
> 
> 추가로 권장하는 과목
> 교양 과목:
> 
> 추가로 필요한 학점은 없습니다.
> 
> 전공 과목:
> 
> 전공 필수: 졸업 종합평가 (2025년 1학기에 필수 이수)
> 전공 선택: 현재 수강 중인 과목들 이외에도 졸업 전에 전공과 관련된 실습과 프로젝트를 진행하는 과목들을 추천드립니다.
> 부전공 과목:
> 
> 추가적인 소프트웨어 관련 과목들을 계속해서 이수하는 것을 권장드립니다.
> 이제 41 학점을 더 채우기 위한 구체적인 계획을 세우는 것이 좋겠습니다. 매 학기 18-21 학점씩 수강하면, 남은 2-3학기 내 졸업이 가능할 것입니다.
> 
> ## 조기 졸업 계획
> 조기 졸업을 위해서는 다음과 같은 계획을 수립하시는 것이 좋습니다:
> 매 학기 18-21 학점씩 수강하여, 남은 2학기 내에 졸업 가능합니다.
> 
> - 2024년 2학기: 21 학점 (과목명, 과목명, 과목명, 과목명 과목 수강)
> 
> - 2025년 1학기: 20 학점 (과목명, 과목명, 과목명, 과목명 과목 수강)
> 
> ## 일반 졸업 계획
> 일반적인 졸업을 위해서는 다음과 같은 계획을 수립하시는 것이 좋습니다:
> 매 학기 15-18 학점씩 수강하여, 남은 학기 내에 졸업 가능합니다.
> 구체적으로는 다음과 같습니다.
> 
> - 2024년 2학기: 18 학점 (과목명, 과목명, 과목명, 과목명 과목 수강)
> 
> - 2025년 1학기: 18 학점 (과목명, 과목명, 과목명, 과목명 과목 수강)
> 
> - 2025년 2학기: 5 학점 (과목명, 과목명, 과목명, 과목명 과목 수강)
> 
> ## 종합
> 현재 진행한 및 계획한 수강 과목들을 잘 이수하면 졸업요건을 충족할 수 있습니다.
> 또한, 졸업종합평가(P/F) 과목을 반드시 심사 시기에 맞춰 이수하세요.
> 지금까지의 성적관리와 학점 이수를 기반으로 충분히 졸업 가능성을 보유하고 있습니다. 앞으로도 성실히 수강 과목들을 완료하시면 무난하게 졸업할 수 있을 것입니다.